In [1]:
import ezdxf
import numpy as np

In [55]:
# Create a new DXF document.
doc = ezdxf.new(dxfversion='R2007')

# Create new table entries (layers, linetypes, text styles, ...).
layer = doc.layers.new('main', dxfattribs={'lineweight': 0.0254})
layer.rgb = (255, 0, 0)

# DXF entities (LINE, TEXT, ...) reside in a layout (modelspace, 
# paperspace layout or block definition).  
msp = doc.modelspace()

In [56]:
# Define constants
WIDTH = 140 # mm
HEIGHT = 200 # mm
n_x = 14
n_y = 20
BUFFER_HEIGHT = 20 # mm, extra length on end to use as a handle
seamhole_diameter = 3 # mm
kerf = 3. # mm
gap = 2*kerf + 1.5 # mm, for defining the straight line segments

unlocked_n_x = 8
locked_n_x = n_x - unlocked_n_x
unlocked_n_y = 0

# Define derived constants
cell_width = WIDTH // n_x
cell_height = HEIGHT // n_y

In [57]:
# Define border
msp.add_line((0,  -BUFFER_HEIGHT), (WIDTH,  -BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})
msp.add_line((0,  -BUFFER_HEIGHT), (0, HEIGHT + BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})
msp.add_line((WIDTH,  -BUFFER_HEIGHT), (WIDTH, HEIGHT + BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})
msp.add_line((0, HEIGHT + BUFFER_HEIGHT), (WIDTH, HEIGHT + BUFFER_HEIGHT), dxfattribs={'lineweight': 0.0254})

<class 'ezdxf.entities.line.Line'> LINE(#31)

In [58]:
def add_arc(center, radius, start_angle, end_angle, is_counter_clockwise=True):
    num_points = 15
    start_angle *= np.pi/180
    end_angle *= np.pi/180
    if is_counter_clockwise:
        theta = np.arange(start_angle, end_angle, abs(end_angle-start_angle)/num_points)
    else:
        theta = np.arange(end_angle, 2*np.pi + start_angle, abs(2*np.pi + start_angle - end_angle)/num_points)
    for n in range(num_points - 1):
        start_pos = (center[0] + radius*np.cos(theta[n]), center[1] + radius*np.sin(theta[n]))
        end_pos = (center[0] + radius*np.cos(theta[n+1]), center[1] + radius*np.sin(theta[n+1]))
        msp.add_line(start_pos, end_pos)     
    
    start_pos = (center[0] + radius*np.cos(theta[-1]), center[1] + radius*np.sin(theta[-1]))
    if is_counter_clockwise:
        end_pos = (center[0] + radius*np.cos(end_angle), center[1] + radius*np.sin(end_angle))
    else:
        end_pos = (center[0] + radius*np.cos(2*np.pi + start_angle), center[1] + radius*np.sin(2*np.pi + start_angle))
    msp.add_line(start_pos, end_pos)     
    

In [59]:
# Define horizontal cuts
for j in range(1, n_y):
    if j % 2 == 1:
        for i in range(unlocked_n_x // 2 + 1):
            for k in [kerf/2, -kerf/2]:
                start_pos = (max(0, cell_width*(2*i-1) + gap/2), cell_height*j+k)
                end_pos = (min(WIDTH, cell_width*(2*i+1) - gap/2), cell_height*j+k)
                msp.add_line(start_pos, end_pos)
            if i != 0: # Add left arc
                center = (max(0, cell_width*(2*i-1) + gap/2), cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=True)
            if i != (n_x // 2): # Add right arc        
                center = (min(WIDTH, cell_width*(2*i+1) - gap/2), cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=False)
    else:
        for i in range(unlocked_n_x // 2):
            for k in [kerf/2, -kerf/2]:
                start_pos = (cell_width*(2*i) + gap/2, cell_height*j+k)
                end_pos = (cell_width*(2*i+2) - gap/2, cell_height*j+k)
                msp.add_line(start_pos, end_pos)
            if True: # Add left arc
                center = (cell_width*(2*i) + gap/2, cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=True)
            if i != (n_x // 2): # Add right arc        
                center = (cell_width*(2*i+2) - gap/2, cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=False)

In [60]:
# Define vertical cuts
for i in range(1, unlocked_n_x):
    if i % 2 == 0:
        for j in range(n_y // 2 + 1):
            for k in [kerf/2, -kerf/2]:
                start_pos = (cell_width*i + k, max(0, cell_height*(2*j-1) + gap/2))
                end_pos = (cell_width*i + k, min(HEIGHT, cell_height*(2*j+1) - gap/2))
                msp.add_line(start_pos, end_pos)
            if True: # Add bottom arc
                center = (cell_width*i, max(0, cell_height*(2*j-1) + gap/2))
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=False)
            if True: # Add top arc        
                center = (cell_width*i, min(HEIGHT, cell_height*(2*j+1) - gap/2))
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=True)
    else:
        for j in range(n_y // 2):
            for k in [kerf/2, -kerf/2]:
                start_pos = (cell_width*i + k, cell_height*(2*j) + gap/2)
                end_pos = (cell_width*i + k, cell_height*(2*j+2) - gap/2)
                msp.add_line(start_pos, end_pos)
            if True: # Add bottom arc
                center = (cell_width*i, cell_height*(2*j) + gap/2)
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=False)
            if True: # Add top arc        
                center = (cell_width*i, cell_height*(2*j+2) - gap/2)
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=True)

Partial lock

In [61]:
# Define horizontal cuts
for j in range(1, unlocked_n_y):
    if j % 2 == 1:
        for i in range(unlocked_n_x // 2 + 1, n_x // 2 + 1):
            for k in [kerf/2, -kerf/2]:
                start_pos = (max(0, cell_width*(2*i-1) + gap/2), cell_height*j+k)
                end_pos = (min(WIDTH, cell_width*(2*i+1) - gap/2), cell_height*j+k)
                msp.add_line(start_pos, end_pos)
            if i != 0: # Add left arc
                center = (max(0, cell_width*(2*i-1) + gap/2), cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=True)
            if i != (n_x // 2): # Add right arc        
                center = (min(WIDTH, cell_width*(2*i+1) - gap/2), cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=False)
    else:
        for i in range(unlocked_n_x // 2, n_x // 2):
            for k in [kerf/2, -kerf/2]:
                start_pos = (cell_width*(2*i) + gap/2, cell_height*j+k)
                end_pos = (cell_width*(2*i+2) - gap/2, cell_height*j+k)
                msp.add_line(start_pos, end_pos)
            if True: # Add left arc
                center = (cell_width*(2*i) + gap/2, cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=True)
            if i != (n_x // 2): # Add right arc        
                center = (cell_width*(2*i+2) - gap/2, cell_height*j)
                add_arc(center, kerf/2, 90, 270, is_counter_clockwise=False)

In [62]:
# Define vertical cuts
for i in range(unlocked_n_x, n_x):
    if i % 2 == 0:
        for j in range(unlocked_n_y // 2 + 1):
            for k in [kerf/2, -kerf/2]:
                start_pos = (cell_width*i + k, max(0, cell_height*(2*j-1) + gap/2))
                end_pos = (cell_width*i + k, min(HEIGHT, cell_height*(2*j+1) - gap/2))
                msp.add_line(start_pos, end_pos)
            if True: # Add bottom arc
                center = (cell_width*i, max(0, cell_height*(2*j-1) + gap/2))
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=False)
            if True: # Add top arc        
                center = (cell_width*i, min(HEIGHT, cell_height*(2*j+1) - gap/2))
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=True)
    else:
        for j in range(unlocked_n_y // 2):
            for k in [kerf/2, -kerf/2]:
                start_pos = (cell_width*i + k, cell_height*(2*j) + gap/2)
                end_pos = (cell_width*i + k, cell_height*(2*j+2) - gap/2)
                msp.add_line(start_pos, end_pos)
            if True: # Add bottom arc
                center = (cell_width*i, cell_height*(2*j) + gap/2)
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=False)
            if True: # Add top arc        
                center = (cell_width*i, cell_height*(2*j+2) - gap/2)
                add_arc(center, kerf/2, 0, 180, is_counter_clockwise=True)

In [63]:
num_points = 30
theta = np.arange(0, 2*np.pi, 2*np.pi/num_points)

In [64]:
# Define seam holes
for i in [cell_width/2, cell_width*3/2, WIDTH - cell_width*3/2, WIDTH - cell_width/2]:
    for j in [cell_height/2 + cell_height*j for j in range(n_y)]:
        for n in range(num_points - 1):
            start_pos = (i + seamhole_diameter*np.cos(theta[n])/2, j + seamhole_diameter*np.sin(theta[n])/2)
            end_pos = (i + seamhole_diameter*np.cos(theta[n+1])/2, j + seamhole_diameter*np.sin(theta[n+1])/2)
            msp.add_line(start_pos, end_pos)     
        start_pos = (i + seamhole_diameter*np.cos(theta[-1])/2, j + seamhole_diameter*np.sin(theta[-1])/2)
        end_pos = (i + seamhole_diameter*np.cos(theta[0])/2, j + seamhole_diameter*np.sin(theta[0])/2)
        msp.add_line(start_pos, end_pos)

In [65]:
# Save DXF document.
doc.saveas('20201130_square_1cm_1.5mm_3mmkerf_fully_locked.dxf')